In [36]:
from __future__ import division
import math
import random
import pandas as pd
 
flowerLables = {0: 'Iris-setosa',
                1: 'Iris-versicolor',
                2: 'Iris-virginica'}
 
random.seed(0)
 
# 生成区间[a, b)内的随机数
def rand(a, b):
    return (b - a) * random.random() + a
 
# 生成大小 I*J 的矩阵，默认零矩阵
def makeMatrix(I, J, fill=0.0):
    m = []
    for i in range(I):
        m.append([fill] * J)
    return m
 
# 函数 sigmoid
def sigmoid(x):
    return 1.0 / (1.0 + math.exp(-x))
 
# 函数 sigmoid 的导数
def dsigmoid(x):
    return x * (1 - x)
 

class NN:
    """ 三层反向传播神经网络 """
 
    def __init__(self, ni, nh, no):
        # 输入层、隐藏层、输出层的节点（数）
        self.ni = ni + 1  # 增加一个偏差节点
        self.nh = nh + 1
        self.no = no
 
        # 激活神经网络的所有节点（向量）
        self.ai = [1.0] * self.ni
        self.ah = [1.0] * self.nh
        self.ao = [1.0] * self.no
 
        # 建立权重（矩阵）
        self.wi = makeMatrix(self.ni, self.nh)
        self.wo = makeMatrix(self.nh, self.no)
        # 设为随机值
        for i in range(self.ni):
            for j in range(self.nh):
                self.wi[i][j] = rand(-0.2, 0.2)
        for j in range(self.nh):
            for k in range(self.no):
                self.wo[j][k] = rand(-2, 2)
 
    def update(self, inputs):
        if len(inputs) != self.ni - 1:
            raise ValueError('与输入层节点数不符！')
 
        # 激活输入层
        for i in range(self.ni - 1):
            self.ai[i] = inputs[i]
 
        # 激活隐藏层
        for j in range(self.nh):
            sum = 0.0
            for i in range(self.ni):
                sum = sum + self.ai[i] * self.wi[i][j]
            self.ah[j] = sigmoid(sum)
 
        # 激活输出层
        for k in range(self.no):
            sum = 0.0
            for j in range(self.nh):
                sum = sum + self.ah[j] * self.wo[j][k]
            self.ao[k] = sigmoid(sum)
 
        return self.ao[:]
 
    def backPropagate(self, targets, lr):
        """ 反向传播 """
 
        # 计算输出层的误差
        output_deltas = [0.0] * self.no
        for k in range(self.no):
            error = targets[k] - self.ao[k]
            output_deltas[k] = dsigmoid(self.ao[k]) * error
 
        # 计算隐藏层的误差
        hidden_deltas = [0.0] * self.nh
        for j in range(self.nh):
            error = 0.0
            for k in range(self.no):
                error = error + output_deltas[k] * self.wo[j][k]
            hidden_deltas[j] = dsigmoid(self.ah[j]) * error
 
        # 更新输出层权重
        for j in range(self.nh):
            for k in range(self.no):
                change = output_deltas[k] * self.ah[j]
                self.wo[j][k] = self.wo[j][k] + lr * change
 
        # 更新输入层权重
        for i in range(self.ni):
            for j in range(self.nh):
                change = hidden_deltas[j] * self.ai[i]
                self.wi[i][j] = self.wi[i][j] + lr * change
 
        # 计算误差
        error = 0.0
        error += 0.5 * (targets[k] - self.ao[k]) ** 2
        return error
 
    def test(self, patterns):
        count = 0
        for p in patterns:
            target = flowerLables[(p[1].index(1))]
            result = self.update(p[0])
            index = result.index(max(result))
            print(p[0], ':', target, '->', flowerLables[index])
            count += (target == flowerLables[index])
        accuracy = float(count / len(patterns))
        print('accuracy: %-.9f' % accuracy)
 
    def weights(self):
        print('输入层权重:')
        for i in range(self.ni):
            print(self.wi[i])
        print()
        print('输出层权重:')
        for j in range(self.nh):
            print(self.wo[j])
 
    def train(self, patterns, iterations=1000, lr=0.1):
        # lr: 学习速率(learning rate)
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backPropagate(targets, lr)
            if i % 100 == 0:
                print('error: %-.9f' % error)
 
 
def iris():
    data = []
    # 读取数据
    path=r'D:\jupyter\人工神经网络\data\iris.txt'
    raw = pd.read_csv(path)
    raw_data = raw.values
    raw_feature = raw_data[0:, 0:4]
    for i in range(len(raw_feature)):
        ele = []
        ele.append(list(raw_feature[i]))
        if raw_data[i][4] == 'Iris-setosa':
            ele.append([1, 0, 0])
        elif raw_data[i][4] == 'Iris-versicolor':
            ele.append([0, 1, 0])
        elif raw_data[i][4]=='Iris-virginica':
            ele.append([0, 0, 1])
        data.append(ele)
    # 随机排列数据
    random.shuffle(data)
    training = data[0:100]
    test = data[101:]
    nn = NN(4, 7, 3)
    print("训练集如下：（前100组数据）\n")
    nn.train(training, iterations=10000)
    print("\n测试级如下：（后50组数据）\n")
    nn.test(test)  

if __name__ == '__main__':
    iris()
    

训练集如下：（前100组数据）

error: 9.721981217
error: 1.748451777
error: 1.508379170
error: 1.493563302
error: 1.559492436
error: 1.605105661
error: 1.647783876
error: 1.688108785
error: 1.707174152
error: 1.687212501
error: 1.646461838
error: 1.585215098
error: 1.493004851
error: 1.386243625
error: 1.213941659
error: 0.737465002
error: 0.690830065
error: 0.683006287
error: 0.696329204
error: 0.715159543
error: 0.758597954
error: 0.839698290
error: 0.783273010
error: 0.694334597
error: 0.613839510
error: 0.590218735
error: 0.578748456
error: 0.572903355
error: 0.569451988
error: 0.566986837
error: 0.564731378
error: 0.561947879
error: 0.557480295
error: 0.548997844
error: 0.536724212
error: 0.528822512
error: 0.524318847
error: 0.521330422
error: 0.519140344
error: 0.517434181
error: 0.516050237
error: 0.514895289
error: 0.513910954
error: 0.513058329
error: 0.512310221
error: 0.511646889
error: 0.511053561
error: 0.510518896
error: 0.510034009
error: 0.509591811
error: 0.509186559
error: 0.50881